# Study - Comparison with Baseline methods 

<!-- All results are saved in `GD4PS/notebooks/rq3_results/[current_time]_results.md` -->

### Methodology 
1. Initialise the dataset. 
2. Initialise the following models: FCNN, GCNN+GAT, GAT, Linegraph Laplacian and SEGNN. 
3. Performance for SE: Train these models on Net 42-A, evaluate the results. 
4. Performance for Generalisability: Train these models on Net 42-B, deploy on Net 42-A, evaluate the results. 
5. Performance for Scalability: Train these models on MVO, evaluate the results. 

In [1]:
import time
import torch 
import os 
import sys 

# to access the models and utils 
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

from src.dataset.custom_dataset import NodeEdgeTapDatasetV2 
from src.training.trainer import trainer 
from utils.model_utils import initialize_model, get_eval_results 
from utils.gen_utils import dataset_splitter, get_device, load_config, plot_va_predictions_notebook
from utils.ppnet_utils import initialize_network 
from utils.load_data_utils import load_sampled_input_data 
from src.model.graph_model import FCNNRegressor

# for consistency among all the models 
torch.manual_seed(0) 

### Train and evaluate all five models

In [16]:
config['data']['load_std'] = 0.3
net_MVO = initialize_network(net_name='MVO', else_load=config['data']['load_std'], verbose=True)

# config['data']['num_samples'] = 80

sampled_input_data_MVO = load_sampled_input_data(sc_type=config['data']['scenario_type'], 
                                                        net=net_MVO, 
                                                        num_samples=config['data']['num_samples'],
                                                        noise=config['data']['noise'],
                                                        trafo_ids=[], # foolproof
                                                        scaler=config['data']['scaler'],
                                                        )

dataset_MVO = NodeEdgeTapDatasetV2(model_name=config['model']['name'], sampled_input_data=sampled_input_data_MVO)

all_loaders_MVO, plot_loader_MVO = dataset_splitter(dataset_MVO,
                                    batch_size=config['loader']['batch_size'], 
                                    split_list=config['loader']['split_list'])

fcnn_dataset_MVO = FCNNDataset(sampled_input_data=sampled_input_data_MVO)

fcnn_all_loaders_MVO, fcnn_plot_loader_MVO = dataset_splitter_fcnn(fcnn_dataset_MVO,
                                            batch_size=config['loader']['batch_size'],
                                            split_list=config['loader']['split_list'])

num_nodes = len(net_MVO.bus.index)

#######################################################################################################
fcnn_model_MVO = FCNNRegressor(in_feat = num_node_features * num_nodes, 
                           hid_feat_list=[128], 
                           out_feat=num_y_node_features * num_nodes)

optimizer_fcnn_MVO = torch.optim.Adam(fcnn_model_MVO.parameters(), 
                                  lr=config['training']['lr'],
                                  weight_decay=0.01,   
                                )
schedular_fcnn_MVO = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_fcnn_MVO,
                                                            mode='min',
                                                            factor=0.1, 
                                                            min_lr=1e-4,
                                                          )


#######################################################################################################
model_NGAT_MVO = initialize_model(model_name="NGATRegressor",
                            dataset=dataset_MVO,
                            node_out_features=config['model']['node_out_features'],
                            list_node_hidden_features=config['model']['list_node_hidden_features'],
                            k_hop_node=config['model']['k_hop_node'],
                            edge_out_features=config['model']['edge_out_features'], 
                            list_edge_hidden_features=config['model']['list_edge_hidden_features'],
                            k_hop_edge=config['model']['k_hop_edge'],
                            trafo_hop=config['model']['trafo_hop'],
                            edge_index_list=sampled_input_data['edge_index'],
                            gat_out_features=config['model']['gat_out_features'],
                            gat_head=config['model']['gat_head'],
                            bias=config['model']['bias'], 
                            normalize=config['model']['normalize'], 
                            device=device,  
                            ).to(device)    

optimizer_NGAT_MVO = torch.optim.Adam(model_NGAT_MVO.parameters(),lr=config['training']['lr'], weight_decay=config['training']['weight_decay'])

schedular_NGAT_MVO = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_NGAT_MVO, 
                                                    mode='min',
                                                    factor=0.1, 
                                                    # patience=1,
                                                    min_lr=config['training']['schedular_min_lr'])

#######################################################################################################
model_GAT_MVO = initialize_model(model_name="GATRegressor",
                            dataset=dataset_MVO,
                            node_out_features=config['model']['node_out_features'],
                            list_node_hidden_features=config['model']['list_node_hidden_features'],
                            k_hop_node=config['model']['k_hop_node'],
                            edge_out_features=config['model']['edge_out_features'], 
                            list_edge_hidden_features=config['model']['list_edge_hidden_features'],
                            k_hop_edge=config['model']['k_hop_edge'],
                            trafo_hop=config['model']['trafo_hop'],
                            edge_index_list=sampled_input_data['edge_index'],
                            gat_out_features=config['model']['gat_out_features'],
                            gat_head=config['model']['gat_head'],
                            bias=config['model']['bias'], 
                            normalize=config['model']['normalize'], 
                            device=device,  
                            ).to(device)    

optimizer_GAT_MVO = torch.optim.Adam(model_GAT_MVO.parameters(),lr=config['training']['lr'], weight_decay=config['training']['weight_decay'])

schedular_GAT_MVO = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_GAT_MVO, 
                                                    mode='min',
                                                    factor=0.1, 
                                                    # patience=1,
                                                    min_lr=config['training']['schedular_min_lr'])

#######################################################################################################
model_LGL_MVO = initialize_model(model_name="NEGATRegressor_LGL",
                            dataset=dataset_MVO,
                            node_out_features=config['model']['node_out_features'],
                            list_node_hidden_features=config['model']['list_node_hidden_features'],
                            k_hop_node=config['model']['k_hop_node'],
                            edge_out_features=config['model']['edge_out_features'], 
                            list_edge_hidden_features=config['model']['list_edge_hidden_features'],
                            k_hop_edge=config['model']['k_hop_edge'],
                            trafo_hop=config['model']['trafo_hop'],
                            edge_index_list=sampled_input_data['edge_index'],
                            gat_out_features=config['model']['gat_out_features'],
                            gat_head=config['model']['gat_head'],
                            bias=config['model']['bias'], 
                            normalize=config['model']['normalize'], 
                            device=device,  
                            ).to(device)

optimizer_LGL_MVO = torch.optim.Adam(model_LGL_MVO.parameters(),lr=config['training']['lr'], weight_decay=config['training']['weight_decay'])

schedular_LGL_MVO = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_LGL_MVO, 
                                                    mode='min',
                                                    factor=0.1,
                                                    min_lr=config['training']['schedular_min_lr'])

#######################################################################################################
model_MVO = initialize_model(model_name=config['model']['name'],
                            dataset=dataset_MVO,
                            node_out_features=config['model']['node_out_features'],
                            list_node_hidden_features=config['model']['list_node_hidden_features'],
                            k_hop_node=config['model']['k_hop_node'],
                            edge_out_features=config['model']['edge_out_features'], 
                            list_edge_hidden_features=config['model']['list_edge_hidden_features'],
                            k_hop_edge=config['model']['k_hop_edge'],
                            trafo_hop=config['model']['trafo_hop'],
                            edge_index_list=sampled_input_data['edge_index'],
                            gat_out_features=config['model']['gat_out_features'],
                            gat_head=config['model']['gat_head'],
                            bias=config['model']['bias'], 
                            normalize=config['model']['normalize'], 
                            device=device,
                            ).to(device)
                            
optimizer_MVO = torch.optim.Adam(model_MVO.parameters(),lr=config['training']['lr'], weight_decay=config['training']['weight_decay'])

schedular_MVO = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_MVO, 
                                                    mode='min',
                                                    factor=0.1, 
                                                    # patience=1,
                                                    min_lr=config['training']['schedular_min_lr'])

#######################################################################################################
#######################################################################################################
#######################################################################################################
all_losses_FCNN_MVO = trainer(model=fcnn_model_MVO, 
                            train_loader=fcnn_all_loaders_MVO[0],
                            val_loader=fcnn_all_loaders_MVO[1],
                            test_loader=fcnn_all_loaders_MVO[2],
                            optimizer=optimizer_fcnn_MVO, 
                            schedular=schedular_fcnn_MVO, 
                            num_epoch=config['training']['num_epochs'],
                            early_stopping=config['training']['early_stopping'],
                            val_patience=config['training']['val_patience'], 
                            device=device,
                            )

all_losses_NGAT_MVO = trainer(model=model_NGAT_MVO, 
                    train_loader=all_loaders_MVO[0], 
                    val_loader=all_loaders_MVO[1], 
                    test_loader=all_loaders_MVO[2], 
                    optimizer=optimizer_NGAT_MVO,
                    schedular=schedular_NGAT_MVO,
                    num_epoch=config['training']['num_epochs'],
                    early_stopping=config['training']['early_stopping'],
                    val_patience=config['training']['val_patience'],  
                    device=device)

all_losses_GAT_MVO = trainer(model=model_GAT_MVO, 
                    train_loader=all_loaders_MVO[0], 
                    val_loader=all_loaders_MVO[1], 
                    test_loader=all_loaders_MVO[2], 
                    optimizer=optimizer_GAT_MVO,
                    schedular=schedular_GAT_MVO,
                    num_epoch=config['training']['num_epochs'],
                    early_stopping=config['training']['early_stopping'],
                    val_patience=config['training']['val_patience'],  
                    device=device)

all_losses_MVO = trainer(model=model_LGL_MVO, 
                    train_loader=all_loaders_MVO[0], 
                    val_loader=all_loaders_MVO[1], 
                    test_loader=all_loaders_MVO[2], 
                    optimizer=optimizer_LGL_MVO,
                    schedular=schedular_LGL_MVO,
                    num_epoch=config['training']['num_epochs'],
                    early_stopping=config['training']['early_stopping'],
                    val_patience=config['training']['val_patience'],  
                    device=device)

all_losses_MVO = trainer(model=model_MVO, 
                    train_loader=all_loaders_MVO[0], 
                    val_loader=all_loaders_MVO[1], 
                    test_loader=all_loaders_MVO[2], 
                    optimizer=optimizer_MVO,
                    schedular=schedular_MVO,
                    num_epoch=config['training']['num_epochs'],
                    early_stopping=config['training']['early_stopping'],
                    val_patience=config['training']['val_patience'],  
                    device=device)


Transformer Indices for MVO are available from [0,141] 

Number of Trafos = 143 
 
Network: MVO is selected 

Net MVO has 320 nodes and 318 edges. 

Filling nan as 0 in tap_pos, tap_neutral, tap_step_degree
Scaling inputs...
Number of V, P measurements 301 out of 640

Number of P_to, Q_to, P_from, Q_from measurements 1788 out of 1944

Dataset for NEGATRegressor selected!


 Directed power flows accounted in dataset...


 get_edge_index_lu handling dictionary of tensors...

At epoch: 0, 	 training loss: 3.050e+00,                     	 validation loss: 2.106e+00 	 lr: 1.00e-02 	 grad_norm: 8.451e+00
At epoch: 1, 	 training loss: 1.989e+00,                     	 validation loss: 9.364e-01 	 lr: 1.00e-02 	 grad_norm: 6.001e+00
At epoch: 2, 	 training loss: 9.147e-01,                     	 validation loss: 9.756e-01 	 lr: 1.00e-02 	 grad_norm: 1.079e+00
At epoch: 3, 	 training loss: 9.525e-01,                     	 validation loss: 9.770e-01 	 lr: 1.00e-02 	 grad_norm: 4.956e-01
At epoch: 

### Scalability results

In [17]:
####################### FCNN Model #####################################
results_fcnn_MVO = get_eval_results(test_loader=fcnn_all_loaders_MVO[2],
                            tap_weight=config['training']['loss_tap_weight'], 
                            trained_model=fcnn_model_MVO, 
                            scaler=sampled_input_data['scaler_y_label'], 
                            fcnn=True, 
                            num_nodes=320)

####################### GCNN+GAT #####################################
results_ngat_MVO = get_eval_results(test_loader=all_loaders_MVO[2],
                            tap_weight=config['training']['loss_tap_weight'], 
                            trained_model=model_NGAT_MVO, 
                            scaler=sampled_input_data['scaler_y_label'])

####################### GAT #####################################
results_gat_MVO = get_eval_results(test_loader=all_loaders_MVO[2],
                            tap_weight=config['training']['loss_tap_weight'], 
                            trained_model=model_GAT_MVO, 
                            scaler=sampled_input_data['scaler_y_label'])

####################### NEGAT_LG #####################################
results_lg_MVO = get_eval_results(test_loader=all_loaders_MVO[2],
                            tap_weight=config['training']['loss_tap_weight'], 
                            trained_model=model_LGL_MVO, 
                            scaler=sampled_input_data['scaler_y_label'])

####################### NEGAT Regressor Model #####################################
results_MVO = get_eval_results(test_loader=all_loaders_MVO[2],
                            tap_weight=config['training']['loss_tap_weight'], 
                            trained_model=model_MVO, 
                            scaler=sampled_input_data['scaler_y_label'])

Calculating results for StandardScaled Voltage and Angles.
Calculating results for StandardScaled Voltage and Angles.
Calculating results for StandardScaled Voltage and Angles.
Calculating results for StandardScaled Voltage and Angles.


In [19]:
def print_results(name, results):
    """Simple function to print results nicely"""
    print(f"\n{'='*50}")
    print(f"{name} Results:")
    print('='*50)
    
    for metric, value in results.items():
        print(f"{metric:25}: {value}")
        print()


print_results("FCNN MVO", results_fcnn_MVO)
print_results("GCNN+GAT MVO", results_ngat_MVO)  
print_results("GAT MVO", results_gat_MVO)        
print_results("LG MVO", results_lg_MVO)          
print_results("NEGAT MVO", results_MVO)


FCNN MVO Results:
Batchwise Average Test Loss: 1.009589e-01

RMSE_V                   : 8.194978e-04

RMSE_A                   : 1.202922e+00

MAE_V                    : 5.505631e-04

MAE_A                    : 9.452806e-01

MaxAE_V                  : 5.174160e-03

MaxAE_A                  : 4.803596e+00

NRMSE_V                  : 7.462014e-05

NRMSE_A                  : -2.231518e-02


GCNN+GAT MVO Results:
Batchwise Average Test Loss: 4.539704e-01

RMSE_V                   : 4.331977e-04

RMSE_A                   : 8.093555e+00

MAE_V                    : 3.318278e-04

MAE_A                    : 5.154775e+00

MaxAE_V                  : 3.573537e-03

MaxAE_A                  : 2.960396e+01

NRMSE_V                  : 3.943725e-05

NRMSE_A                  : -1.511558e-01


GAT MVO Results:
Batchwise Average Test Loss: 7.715740e-01

RMSE_V                   : 2.432713e-04

RMSE_A                   : 1.154641e+01

MAE_V                    : 1.957112e-04

MAE_A                    : 9.7